<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Document Classification
## *Data Science Unit 4 Sprint 1 Lesson 3*

Today's lesson will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*). 

Today's all about having fun and practicing your skills.

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

## Text Feature Extraction & Classification Pieplines
<a id="p1"></a>

In [46]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

# categories = ['alt.atheism',
#               'talk.religion.misc']

import pandas as pd

sample_sub = pd.read_csv('ds4-which-whisk/sample_submission.csv')
test = pd.read_csv('ds4-which-whisk/test.csv')
train = pd.read_csv('ds4-which-whisk/sample_submission.csv')
test.head()

,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [52]:
train.head()

,id,category
0,955,1
1,3532,3
2,1390,2
3,1024,4
4,1902,2


In [53]:
df = pd.merge(train, test, on='id', how='outer')
df.head()

,id,category,author,description,price,ratingValue,pert_alcohol
0,955,1,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,3,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,2,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,4,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,2,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [60]:
# Tokenizer Pipe

tokens = []

""" Make them tokens """
for doc in tokenizer.pipe(df['reviews.text'], batch_size=500):#similar to scikit learn pipeline
    doc_tokens = [token.text for token in doc]
    tokens.append(doc_tokens)
#     doc_tokens = []
#     for token in doc:
#         if (token.is_stop == False) & (token.is_punct == False):
#             doc_tokens.append(token.text.lower())
            
#    tokens.append(doc_tokens)
    
df['tokens'] = tokens

In [47]:
#data

data = test['description']

#fetch_20newsgroups(subset='train', categories=categories)

#### Sklearn Pipeline Objects

In [48]:
# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier#stochastic gradient descent
from sklearn.feature_extraction.text import TfidfVectorizer


In [49]:
# Create Pipeline

vect = TfidfVectorizer(stop_words='english')#instantiate vectorizer
sgdc = SGDClassifier()#instantiate classifier

pipe = Pipeline([('vect', vect), ('clf', sgdc)])#put objects in pipe#the last item in pipeline, should have fit command

In [54]:
# Fit Pipeline
#pipe.fit(data.data, data.target)
pipe.fit(df['description'], df['category'])

C:\Users\lilyx\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [55]:
pipe.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([2, 3], dtype=int64)

#### Tuning a Pipeline Object with GridSearch

In [56]:
# Experiment Management
from sklearn.model_selection import GridSearchCV

In [86]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),#word appears than more than 50% of documents, excluded, 
    #max df is a parameter of tfidf
    #vect__min_df works similarly must be inside at least 2% 
    #'vect__min_df': (.02, .05),
    'vect__max_features':(500, 500, 1000),#total number of words tfidf will learn after above parameters
    'clf__max_iter':(2, 10, 10)#number of times makes a pass search 
    #max_iter SGDClassifier
}

In [87]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)

In [88]:
grid_search.fit(df['description'], df['category'])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter vect for estimator Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
sample_sub.head(2)

In [ ]:
test['category'] = pipe.predict(test['description'])
test.head(2)

In [ ]:
submission = test[['id', 'category']]
submission.head(2)

In [ ]:
submission.to_csv('Wsubmission01.csv', index = False)

#### Latent Semantic Indexing
<a id="p2"></a>

In [ ]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, #take features and reduce to how many features
                   algorithm='randomized',#how svd is working, randomized is best bet, 
                   n_iter=10)#number of passes at estimating components

In [ ]:
# LSI

lsi = Pipeline([('vect', vect), ('svd', svd)])

In [ ]:
# Pipe

pipe = Pipeline([('lsi', lsi), ('clf', sgdc)])

params = {
    'lsi__vect__max_df': (10, 10, 10)#least indexing
}

In [ ]:
grid_search = GridSearchCV(pipe,parameters, cv=15, n_jobs=5, verbose=1)

In [ ]:
# Fit
pipe.fit(df['description'], df['category'])
test['category'] = pipe.predict(test['description'])
submission = test[['id', 'category']]
submission.head(2)

In [ ]:
submission.to_csv('Wsubmission02.csv', index = False)

#### Word Embeddings with Spacy
<a id="p3"></a>

In [93]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
# doc = nlp("Two bananas in pyjamas")

In [89]:
# bananas_vector = doc.vector
# print(len(bananas_vector))

NameError: name 'doc' is not defined

In [94]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [95]:
X = get_word_vectors(df['description'])

sgdc.fit(X, df['category'])

C:\Users\lilyx\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [102]:
test['category'] = sgdc.predict(get_word_vectors(test['description']))
submission = test[['id', 'category']]
submission.head(2)

,id,category
0,955,1
1,3532,1


In [103]:
submission.to_csv('Wsubmission03.csv', index = False)